# Foundations

## Building a Neural Net Layer from Scratch

### Modeling a Neuron

In [52]:
import numpy as np

In [53]:
weight = np.random.randn(100)
inp = np.random.randn(100) * 100
bias = np.array([.2])

In [54]:
output = sum([x*w for x,w in zip(inp,weight)]) + bias

In [55]:
output

array([-645.06698031])

In [56]:
# Activation

def relu(x): 
  return x if x >= 0 else 0

relu(output), relu(-343)

(0, 0)

In [57]:
y = inp @ weight.T + bias
y

array([-645.06698031])

### Matrix Multiplication from Scratch

In [58]:
import torch
from torch import tensor

In [59]:
def matmul(a,b):
    ar,ac = a.shape # n_rows * n_cols
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac): c[i,j] += a[i,k] * b[k,j]
    return c

In [60]:
m1 = torch.randn(5, 784)
m2 = torch.randn(784,10)

In [61]:
%time t1=matmul(m1, m2)

CPU times: user 619 ms, sys: 0 ns, total: 619 ms
Wall time: 626 ms


In [62]:
%time t1=m1 @ m2

CPU times: user 1.45 ms, sys: 0 ns, total: 1.45 ms
Wall time: 1.02 ms


### Elementwise Arithmetic

In [63]:
a = tensor([10., 6, -4])
b = tensor([2., 8, 7])
a + b

tensor([12., 14.,  3.])

In [64]:
a < b

tensor([False,  True,  True])

In [65]:
(a < b).all(), (a == b).all()

(tensor(False), tensor(False))

In [66]:
(a == a).all()

tensor(True)

In [67]:
(a + b).mean(), (a + b).mean().item()

(tensor(9.6667), 9.666666984558105)

In [68]:
m = tensor([
           [1., 2, 3], 
            [4,5,6], 
            [7,8,9]])
m*m

tensor([[ 1.,  4.,  9.],
        [16., 25., 36.],
        [49., 64., 81.]])

In [69]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc): c[i,j] = (a[i] * b[:,j]).sum()
    return c

In [70]:
%timeit -n 20 t3 = matmul(m1,m2)

20 loops, best of 5: 829 µs per loop


### Boardcasting

#### Broadcasting with a scalar

In [74]:
a = tensor([10., 4, -4])
a > 0

tensor([ True,  True, False])

In [75]:
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])
(m - 5) / 2.73

tensor([[-1.4652, -1.0989, -0.7326],
        [-0.3663,  0.0000,  0.3663],
        [ 0.7326,  1.0989,  1.4652]])

#### Broadcasting a vector to a matrix

In [76]:
c = tensor([10., 20, 40])
m = tensor([[1., 2, 3], [4, 5, 6], [7, 8, 9]])

m.shape, c.shape

(torch.Size([3, 3]), torch.Size([3]))

In [79]:
m + c

tensor([[11., 22., 43.],
        [14., 25., 46.],
        [17., 28., 49.]])

In [80]:
c.expand_as(m)

tensor([[10., 20., 40.],
        [10., 20., 40.],
        [10., 20., 40.]])

In [81]:
t = c.expand_as(m)

t.storage()

 10.0
 20.0
 40.0
[torch.FloatStorage of size 3]

In [83]:
t.stride(), t.shape

((0, 1), torch.Size([3, 3]))

In [84]:
c + m

tensor([[11., 22., 43.],
        [14., 25., 46.],
        [17., 28., 49.]])

In [85]:
c = tensor([10.,20,30])
m = tensor([[1., 2, 3], [4,5,6]])

In [86]:
c + m

tensor([[11., 22., 33.],
        [14., 25., 36.]])

In [87]:
# This won't work

c = tensor([10.,20])
m = tensor([[1., 2, 3], [4,5,6]])
c+m

RuntimeError: ignored

In [88]:
c = tensor([10.,20,30])
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])

In [92]:
c = c.unsqueeze(1)
c.shape

torch.Size([3, 1])

In [93]:
c + m

tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 38., 39.]])

In [94]:
c = tensor([10.,20,30])
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])

c + m

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

In [95]:
c = tensor([10, 20, 30])

c.shape, c.unsqueeze(0).shape, c.unsqueeze(1).shape

(torch.Size([3]), torch.Size([1, 3]), torch.Size([3, 1]))

In [99]:
c = tensor([10.,20,30])
m = tensor([[1., 2, 3], 
            [4,5,6], 
            [7,8,9]])
c = c.unsqueeze(1)
c + m

tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 38., 39.]])

In [96]:
c = tensor([10.,20,30])
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])
c = c.unsqueeze(1)

c + m

tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 38., 39.]])

In [100]:
c.shape, c[None, :].shape, c[:, None].shape

(torch.Size([3, 1]), torch.Size([1, 3, 1]), torch.Size([3, 1, 1]))

In [101]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
#       c[i,j] = (a[i,:]          * b[:,j]).sum() # previous
        c[i]   = (a[i  ].unsqueeze(-1) * b).sum(dim=0)
    return c

In [102]:
%timeit -n 20 t4 = matmul(m1,m2)

20 loops, best of 5: 167 µs per loop


### Einstein Summation

In [109]:
def matmul(a, b):
  return torch.einsum("ik,kj->ij", a, b)

In [112]:
matmul(c, m)

tensor([[120., 150., 180.],
        [240., 300., 360.],
        [360., 450., 540.]])

## The Forward and Backward Passes

#### Defining and Initializing a Layer

In [113]:
def lin(x, w, b):
  return x @ w + b

In [114]:
x = torch.randn(200, 100)
y = torch.randn(200)

In [117]:
w1 = torch.randn(100, 50)
b1 = torch.zeros(50)
w2 = torch.randn(50, 1)
b2 = torch.zeros(1)

In [119]:
li = lin(x, w1, b1)
li.shape

torch.Size([200, 50])

In [121]:
li.mean(), li.std()

(tensor(-0.0168), tensor(9.9339))

In [124]:
x = torch.randn(200, 100)

for i in range(50):
  x = x @ torch.randn(100, 100)

x[0:5, 0:5]

tensor([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]])

In [125]:
x = torch.randn(200, 100)
for i in range(50): x = x @ (torch.randn(100,100) * 0.01)
x[0:5,0:5]

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [127]:
x = torch.randn(200, 100)
for i in range(50): x = x @ (torch.randn(100,100) * 0.1)
x[0:5,0:5]

tensor([[ 0.6465,  0.4784,  0.5942, -0.6780, -0.0538],
        [-0.2086,  0.3057, -0.0281,  0.5179, -0.3318],
        [ 0.6698,  0.2750, -0.1796, -0.7969,  0.1941],
        [ 0.2625, -0.6445, -1.2116, -1.6199,  0.0021],
        [-0.0277,  0.1543, -0.5734, -0.5095,  0.5634]])

In [128]:
x.std()

tensor(1.0011)

In [135]:
x = torch.randn(200, 100)
y = torch.randn(200)

In [136]:
from math import sqrt

w1 = torch.randn(100,50) / sqrt(100)
b1 = torch.zeros(50)
w2 = torch.randn(50,1) / sqrt(50)
b2 = torch.zeros(1)

In [137]:
l1 = lin(x, w1, b1)
l1.mean(),l1.std()

(tensor(0.0119), tensor(0.9839))

In [138]:
def relu(x):
  return x.clamp_min(0.)

In [139]:
l2 = relu(l1)

l2.mean(), l2.std()

(tensor(0.3954), tensor(0.5791))

In [140]:
x = torch.randn(200, 100)
for i in range(50): x = relu(x @ (torch.randn(100,100) * 0.1))
x[0:5,0:5]

tensor([[5.9772e-09, 3.7373e-09, 3.5192e-08, 0.0000e+00, 0.0000e+00],
        [1.2636e-09, 9.9859e-09, 3.3379e-08, 4.0781e-09, 0.0000e+00],
        [0.0000e+00, 4.6233e-09, 1.5527e-08, 1.9373e-09, 0.0000e+00],
        [9.3246e-09, 8.8139e-09, 5.2562e-08, 1.1667e-09, 0.0000e+00],
        [0.0000e+00, 3.4275e-09, 1.1248e-08, 1.4312e-09, 0.0000e+00]])

In [141]:
x = torch.randn(200, 100)
for i in range(50): x = relu(x @ (torch.randn(100,100) * sqrt(2/100)))
x[0:5,0:5]

tensor([[1.9405, 1.3098, 0.2031, 1.8069, 0.0000],
        [1.7460, 1.2284, 0.1835, 1.7792, 0.0000],
        [1.7311, 1.1862, 0.1931, 1.5623, 0.0000],
        [1.6057, 0.9925, 0.1110, 1.3846, 0.0000],
        [2.1139, 1.3955, 0.1781, 1.8758, 0.0000]])

In [142]:
x = torch.randn(200, 100)
y = torch.randn(200)

w1 = torch.randn(100,50) * sqrt(2 / 100)
b1 = torch.zeros(50)
w2 = torch.randn(50,1) * sqrt(2 / 50)
b2 = torch.zeros(1)

In [143]:
l1 = lin(x, w1, b1)
l2 = relu(l1)
l2.mean(), l2.std()

(tensor(0.5549), tensor(0.8259))

In [144]:
def model(x):
    l1 = lin(x, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [145]:
out = model(x)
out.shape

torch.Size([200, 1])

In [146]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [147]:
loss = mse(out, y)

#### Gradients and the Backward Pass



```
loss = mse(out, y) = mse(lin(l2, w2, b2), y)
```



In [154]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [155]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [156]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g
    b.g = out.g.sum(0)

In [157]:
from sympy import symbols, diff

In [158]:
sx, sy = symbols('sx sy')

In [159]:
diff(sx ** 2, sx)

2*sx

In [160]:
diff(sx / 43, sx)

1/43

In [162]:
def forward_and_backward(inp, targ):
  #forward
  l1 = inp @ w1 + b1
  l2 = relu(l1)
  out = l2 @ w2 + b2

  loss = mse(out, targ)

  # backward

  mse_grad(out, targ)
  lin_grad(l2, out, w2, b2)
  relu_grad(l1, l2)
  lin_grad(inp, l1, w1, b1)

#### Refactoring the Model

In [164]:
class Relu():

  def __call__(self, inp):
    self.inp = inp
    self.out = inp.clamp_min(0.)
    return self.out
  def backward(self):
    self.inp.g = (self.inp > 0).float() * self.out.g

In [167]:
class Lin():
  def __init__(self, w, b):
    self.w, self.b = w, b

  def __call__(self, inp):
    self.inp = inp
    self.out = inp @ self.w + self.b
    return self.out
  
  def backward(self):
    self.inp.g = self.out.g @ self.w.t()
    self.w.g = self.inp.t() @ self.out.g
    self.b.g = self.out.g.sum(0)

In [168]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        x = (self.inp.squeeze()-self.targ).unsqueeze(-1)
        self.inp.g = 2.*x/self.targ.shape[0]

In [169]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [170]:
model = Model(w1, b1, w2, b2)

In [171]:
loss = model(x, y)

In [172]:
model.backward()

#### Going to PyTorch

In [173]:
class LayerFunction():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self):  raise Exception('not implemented')
    def bwd(self):      raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [174]:
class Relu(LayerFunction):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [175]:
class Lin(LayerFunction):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = out.g.sum(0)

In [176]:
class Mse(LayerFunction):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): 
        inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [177]:
from torch.autograd import Function

class MyRelu(Function):
    @staticmethod
    def forward(ctx, i):
        result = i.clamp_min(0.)
        ctx.save_for_backward(i)
        return result
    
    @staticmethod
    def backward(ctx, grad_output):
        i, = ctx.saved_tensors
        return grad_output * (i>0).float()

In [178]:
import torch.nn as nn

class LinearLayer(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(n_out, n_in) * sqrt(2/n_in))
        self.bias = nn.Parameter(torch.zeros(n_out))
    
    def forward(self, x): return x @ self.weight.t() + self.bias

In [179]:
lin = LinearLayer(10,2)
p1,p2 = lin.parameters()
p1.shape,p2.shape

(torch.Size([2, 10]), torch.Size([2]))

In [180]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out))
        self.loss = mse
        
    def forward(self, x, targ): return self.loss(self.layers(x).squeeze(), targ)